## Third Movement: 

In [314]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [315]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [316]:
%run attack_graph_MIR100.ipynb

### Use Third Distribution

In [317]:
# The key change - enhanced random_steps function
attack_rate_list = [0]   # Not used but kept for API consistency
defense_rate_list = [0]  # Not used but kept for API consistency

In [318]:
# Define the random_steps function with proportional probability handling
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    print(f"\nDEBUG - Processing route: {route}")
    
    # Collect edge probabilities
    hardness = []
    for i in range(len(route) - 1):
        edge_data = graph[route[i]][route[i + 1]]
        print(f"Edge {route[i]}->{route[i + 1]}: full edge data = {edge_data}")
        prob = float(edge_data.get('edge_probabilities', 1.0))
        hardness.append(prob)
    
    hardness = np.array(hardness)
    print(f"Final hardness array: {hardness}")
    
    # Replace NaN with 1.0 (for edges with undefined probabilities)
    hardness = np.nan_to_num(hardness, nan=1.0)
    
    # Compute cumulative product for paths leading to each node
    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness[:-1])))
    print(f"Cumulative probabilities: {cumulative_probs}")
    
    # Calculate stop probabilities, excluding the final position for multiplication
    stop_probs = 1 - hardness
    stop_probs = np.concatenate((stop_probs, [1.0]))
    print(f"Stop probabilities: {stop_probs}")

    # Align cumulative_probs with stop_probs by adding a 1.0 at the end
    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness)))
    print(f"Cumulative probabilities: {cumulative_probs}")

    # Ensure both arrays have the same length
    if cumulative_probs.shape != stop_probs.shape:
        raise ValueError(
            f"Shape mismatch: cumulative_probs {cumulative_probs.shape}, stop_probs {stop_probs.shape}"
        )

    # Calculate raw PDF
    pdf = cumulative_probs * stop_probs
    print(f"Raw PDF: {pdf}")
    
    # Normalize the PDF
    normalized_pdf = pdf / np.sum(pdf)
    print(f"Normalized PDF: {normalized_pdf}")
    
    return normalized_pdf


In [319]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


DEBUG - Processing route: [1, 5, 15, 17]
Edge 1->5: full edge data = {'edge_probabilities': 0.111265, 'weight': array(2.19584054)}
Edge 5->15: full edge data = {'edge_probabilities': 0.111265, 'weight': array(2.19584054)}
Edge 15->17: full edge data = {'edge_probabilities': 0.47287625, 'weight': array(0.74892155)}
Final hardness array: [0.111265   0.111265   0.47287625]
Cumulative probabilities: [1.        0.111265  0.0123799]
Stop probabilities: [0.888735   0.888735   0.52712375 1.        ]
Cumulative probabilities: [1.         0.111265   0.0123799  0.00585416]
Raw PDF: [0.888735   0.0988851  0.00652574 0.00585416]
Normalized PDF: [0.888735   0.0988851  0.00652574 0.00585416]

DEBUG - Processing route: [5, 15, 17]
Edge 5->15: full edge data = {'edge_probabilities': 0.111265, 'weight': array(2.19584054)}
Edge 15->17: full edge data = {'edge_probabilities': 0.47287625, 'weight': array(0.74892155)}
Final hardness array: [0.111265   0.47287625]
Cumulative probabilities: [1.       0.11126

In [320]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Sun Dec 22 18:41:49 2024"

++++++++++++++++++++++++++++++++

The virtual target nodeID is 17

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 2.388661e-01
15 1.976793e-01
5 0.000000e+00
6 0.000000e+00
7 0.000000e+00
8 5.634545e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.3411805
3 0.1376046
4 0.0000000
5 0.0000000
6 0.0000000
7 0.0000000
8 0.0000000
9 0.0000000
10 0.5212149
11 0.0000000
[1] 0.085

Defender can keep attacker success below: 0.085
Attacker can guarantee success probability of: 0.085



In [321]:
# def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
#     """
#     Calculate probability distribution based on edge difficulties.
    
#     Args:
#         route: List of nodes representing an attack path
#         attack_rate: Not used in this implementation but kept for API consistency
#         defense_rate: Not used in this implementation but kept for API consistency
#         graph: NetworkX graph containing edge probabilities
        
#     Returns:
#         numpy array: Probability distribution for number of steps attacker can take
#     """
#     # Get edge probabilities between consecutive nodes in route
#     hardness = []
#     for i in range(len(route)-1):
#         # Get edge data between these nodes
#         edges_data = graph.get_edge_data(route[i], route[i+1])
        
#         # Handle both MultiDiGraph and DiGraph edge data structures
#         if isinstance(edges_data, dict):
#             if 'edge_probabilities' in edges_data:
#                 # DiGraph case - direct dictionary
#                 prob = float(edges_data.get('edge_probabilities', 1.0))
#                 hardness.append(prob)
#             else:
#                 # MultiDiGraph case - dictionary of edge dictionaries
#                 probs = []
#                 for key in edges_data:
#                     if isinstance(edges_data[key], dict):
#                         prob = edges_data[key].get('edge_probabilities', 1.0)
#                         probs.append(float(prob))
#                     else:
#                         probs.append(float(edges_data.get('edge_probabilities', 1.0)))
#                 hardness.append(max(probs) if probs else 1.0)
#         else:
#             # No edge data found
#             hardness.append(1.0)
    
#     # Convert to numpy array and handle missing values
#     hardness = np.array(hardness)
#     hardness[np.isnan(hardness)] = 1.0  # Missing probabilities treated as certainty
    
#     # Calculate PDF according to the paper's formula
#     # P(n steps) = (1-p_{n+1}) * product(p_1 to p_n)
#     # where p_i is probability of success at step i
    
#     # Calculate cumulative products
#     cumprod = np.concatenate(([1.0], np.cumprod(hardness)))
    
#     # Calculate (1-p) terms
#     one_minus_h = np.concatenate((1 - hardness, [1.0]))
    
#     # Final PDF calculation
#     pdf = one_minus_h * cumprod
    
#     # Normalize to ensure it sums to 1
#     pdf = pdf / np.sum(pdf)
    
#     return pdf